In [0]:
CREATE TABLE archived_orders (
    order_id INT,
    order_date DATE,
    customer_id INT,
    amount DECIMAL(10, 2)
);

CREATE TABLE orders (
    order_id INT,
    order_date DATE,
    customer_id INT,
    amount DECIMAL(10, 2)
);

INSERT INTO orders (order_id, order_date, customer_id, amount) VALUES
(1, '2023-05-15', 101, 250.00),
(2, '2023-11-20', 102, 150.00),
(3, '2025-02-10', 103, 300.00),
(4, '2025-04-25', 104, 450.00);

In [0]:
CREATE PROCEDURE archive_old_orders(cut_date DATE)
LANGUAGE SQL
SQL SECURITY INVOKER
AS BEGIN
    INSERT INTO archived_orders
        SELECT * FROM orders WHERE order_date < cut_date;

    DELETE FROM orders WHERE order_date < cut_date;
END;

In [0]:
CALL archive_old_orders(date'2024-01-01')

In [0]:
CREATE TABLE inventory (
    id INT,
    product_name STRING,
    stock INT
);

INSERT INTO inventory (id, product_name, stock) VALUES
(1001, 'Product A', 100),
(1002, 'Product B', 200),
(1003, 'Product C', 150);

In [0]:
CREATE OR REPLACE PROCEDURE update_inventory(product_id INT, quantity INT)
LANGUAGE SQL
SQL SECURITY INVOKER
AS BEGIN
    UPDATE inventory
    SET stock = (stock - update_inventory.quantity)
    WHERE id = update_inventory.product_id;
END;

In [0]:
CALL update_inventory(1001, 20);
CALL update_inventory(1002, 50);

In [0]:
-- SQL Function (Only returns single value)
CREATE FUNCTION calculate_tax(amount DECIMAL(10,2))
RETURNS DECIMAL(10,2)
RETURN amount * 0.08;

SELECT calculate_tax(100); -- Returns 8.00

In [0]:
-- Stored Procedure (Multiple OUT params)
CREATE PROCEDURE calculate_tax_and_total(
    IN amount DECIMAL(10,2),
    OUT tax DECIMAL(10,2),
    OUT total DECIMAL(10,2)
)
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
    SET tax = amount * 0.08;
    SET total = amount + tax;
END;

In [0]:
DECLARE tax DECIMAL(10,2);
DECLARE total DECIMAL(10,2);
CALL calculate_tax_and_total(100, tax, total);
SELECT tax, total;

In [0]:
CREATE OR REPLACE TABLE rate_log (
    rate DECIMAL(10,2),
    log_date DATE,
    source STRING
);

In [0]:
-- Inline Function -- No DML allowed
CREATE OR REPLACE FUNCTION usd_to_eur_func(usd DECIMAL(10,2))
RETURNS DECIMAL(10,2)
RETURN usd * 0.92;

-- can be used inside SELECT
INSERT INTO rate_log (rate, log_date, source)
SELECT usd_to_eur_func(10), current_date(), 'func';

In [0]:
-- Stored Procedure, can include DML bur must be CALLed separately:
CREATE OR REPLACE PROCEDURE usd_to_eur_proc(IN usd DECIMAL(10,2))
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
    DECLARE eur DECIMAL(10,2);
    SET eur = usd * 0.92;
    INSERT INTO rate_log(rate, log_date, source) VALUES(eur, current_date(), 'proc');
END;

In [0]:
-- standalone call
CALL usd_to_eur_proc(10); 

In [0]:
SELECT * FROM rate_log;

In [0]:
CREATE OR REPLACE PROCEDURE validate_card(IN card_number STRING)
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
    DECLARE total INT DEFAULT 0;
    DECLARE pos INT DEFAULT length(card_number);
    DECLARE idx INT DEFAULT 0;
    DECLARE digit INT;
    -- Loop through digits from right (pos = length) to left (pos = 1)
    WHILE pos > 0 DO
        SET digit = CAST(SUBSTRING(card_number, pos, 1) AS INT);
        IF idx % 2 = 1 THEN
            -- Double every second digit and subtract 9 if result >= 10
            SET digit = digit * 2;
            IF digit > 9 THEN 
                SET digit = digit - 9;
            END IF;
        END IF;
        SET total = total + digit;
        SET idx = idx + 1;
        SET pos = pos - 1;
    END WHILE;
    IF total % 10 = 0 THEN
        SELECT 'VALID' AS result;
    ELSE
        SELECT 'INVALID' AS result;
    END IF;
END;


In [0]:
CALL validate_card('3434343434343434')